<a href="https://colab.research.google.com/github/amithadityacp/Mastistis_GenAi/blob/main/Mastits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers accelerate peft datasets bitsandbytes sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 15.3 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "json",
    data_files={"train": "/content/mastitis_cure_training_500 (1).json"
    }
)

print("Sample record:", dataset["train"][0])


Generating train split: 0 examples [00:00, ? examples/s]

Sample record: {'input': 'Milk Temperature: 40.2°C, pH: 7.27, Somatic Cell Count: 519, Conductivity: 5.21, Clotting: 1', 'output': 'The cow shows signs of mild mastitis. Check udder firmness and redness. Apply warm compresses and ensure proper milking hygiene. For mild cases, use veterinary-prescribed antibiotics (e.g., cephapirin or cloxacillin). Provide adequate hydration and isolate the affected quarters to prevent spread.'}


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "EleutherAI/gpt-neo-1.3B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_8bit=True
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

In [ ]:
def tokenize_function(example):
    prompt = example["input"]
    output = example["output"]
    text = f"{prompt}\n{output}"
    return tokenizer(text, truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=False)


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["c_attn", "c_proj"],  # correct GPT-Neo 1.3B modules
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 3,932,160 || all params: 1,319,507,968 || trainable%: 0.2980


In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

# Fix padding token issue
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

# Collator for causal LM
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Training arguments
training_args = TrainingArguments(
    output_dir="./mastitis_model",
    per_device_train_batch_size=2,        # small batch size for Colab
    gradient_accumulation_steps=4,        # accumulate gradients
    learning_rate=2e-4,
    num_train_epochs=5,                   # adjust based on dataset size
    fp16=True,                            # mixed precision
    save_total_limit=2,
    logging_steps=50
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=None,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Start training
trainer.train()


/tmp/ipython-input-718576839.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: amithaditya_23cb04 (amithaditya_23cb04-kgisl-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
50,1.012500
100,0.247500
150,0.239900
200,0.241600
250,0.237700
300,0.236300


TrainOutput(global_step=315, training_loss=0.36273151352292016, metrics={'train_runtime': 647.2691, 'train_samples_per_second': 3.862, 'train_steps_per_second': 0.487, 'total_flos': 1896794523303936.0, 'train_loss': 0.36273151352292016, 'epoch': 5.0})

In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
50,0.240400
100,0.235500
150,0.232400
200,0.235300


Step,Training Loss
50,0.240400
100,0.235500
150,0.232400
200,0.235300
250,0.232500
300,0.230500


TrainOutput(global_step=315, training_loss=0.23411030390905954, metrics={'train_runtime': 487.4541, 'train_samples_per_second': 5.129, 'train_steps_per_second': 0.646, 'total_flos': 1896794523303936.0, 'train_loss': 0.23411030390905954, 'epoch': 5.0})

In [ ]:
# Save your fine-tuned model and tokenizer
model.save_pretrained("./newmastitis_model")
tokenizer.save_pretrained("./newmastitis_model")

print("Model and tokenizer saved successfully at ./newmastitis_model")


Model and tokenizer saved successfully at ./newmastitis_model


In [ ]:
import re
from transformers import pipeline

# Load pipeline with your fine-tuned model
pipe = pipeline(
    "text-generation",
    model="./newmastitis_model",
    tokenizer=tokenizer,
    device=0
)

milk_temp = input("Enter Milk Temperature (°C): ")
ph = input("Enter pH: ")
scc = input("Enter Somatic Cell Count: ")
conductivity = input("Enter Conductivity: ")
clotting = input("Enter Clotting: ")

# Create the dynamic prompt
prompt = f"Milk Temperature: {milk_temp}°C, pH: {ph}, Somatic Cell Count: {scc}, Conductivity: {conductivity}, Clotting: {clotting}. Suggest next step."

# Generate text
result = pipe(
    prompt,
    max_new_tokens=150,
    repetition_penalty=1.2,
    do_sample=True,
    top_p=0.95,
    temperature=0.7
)[0]['generated_text']

# Smart split to handle decimals and "e.g."
lines = re.split(r'(?<!\d)\.(?!\d)|(?<=\be\.g)\.', result)
lines = [line.strip() for line in lines if line.strip()]

# Remove duplicates
seen = set()
clean_lines = []
for line in lines:
    if line not in seen:
        clean_lines.append(line)
        seen.add(line)

# Further split by commas for granular steps
final_lines = []
for line in clean_lines:
    parts = [p.strip() for p in line.split(',') if p.strip()]
    final_lines.extend(parts)

# Print actionable suggestions
print("\nGenerated actionable suggestions:")
for line in final_lines:
    print("-", line)


Device set to use cuda:0


Enter Milk Temperature (°C): 43
Enter pH: 7.27
Enter Somatic Cell Count: 521
Enter Conductivity: 5.19
Enter Clotting: 1

Generated actionable suggestions:
- Milk Temperature: 43°C
- pH: 7.27
- Somatic Cell Count: 521
- Conductivity: 5.19
- Clotting: 1. Suggest next step
- Apply warm compresses
- ensure proper milking hygiene
- and ensure proper post-milking teat dipping
- For severe cases
- use veterinary-prescribed antibiotics (e
- g
- cephapirin or cloxacillin)
- Provide adequate hydration and isolate the affected quarters to prevent spread
- The veterinary-prescribed medicine (e
- cephapirin or cloxacillin) likely needs to be re-prescribed
- Monitor the cow for signs and proper isolate the affected quarters to prevent spread
- The affected cow shows signs of moderate mastitis
- Check udder firmness and redness
- Apply warm compresses and ensure proper milking hygiene
